# Website Summarizer using Ollama locally

Building a website summarizer that uses Llama 3.2 running locally

In [ ]:
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
# Constants
OLLAMA_API = "http://localhost:11434/api/chat" #url on local box
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

ourwebsite = Website("https://espn.com")

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(ourwebsite):
    user_prompt = f"You are looking at a website titled {ourwebsite.title}"
    user_prompt += "\nThe contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += ourwebsite.text
    return user_prompt

In [ ]:
def messages_for(ourwebsite):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(ourwebsite)}
    ]

In [ ]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

# Step 4: print the result
summarize("https://espn.com")

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://espn.com")